In [163]:
import numpy as np
from numpy.linalg import inv

# Ejemplo
## cjt. datos 23, problema PL 1

In [164]:
#matriz de restricciones
A = np.array(
    [[15, -36, 30, -54, 39, -92, 54, -76, 75, 30, 54, -36, -18, 79, 0, 0, 0, 0, 0, 0], 
     [-41, -38, 90, 74, 56, 37, 32, 92, 32, 35, 57, 64, 98, -51, 0, 0, 0, 0, 0, 0], 
     [-48, 38, -18, 20, 89, 15, 56, -64, -78, 35, 65, 55, -87, -23, 0, 0, 0, 0, 0, 0],
     [-65, 36, 77, 49, 74, 68, 56, -91, 37, 78, 9, 63, -36, -63, 0, 0, 0, 0, 0, 0],
     [100, 93, 50, 86, 50, 81, 67, 52, 52, 87, 61, 72, 68, 97, 1, 0, 0, 0, 0, 0],
     [-23, -27, -69, 40, -93, 48, -11, -51, 64, -82, 0, 47, 65, 97, 0, 1, 0, 0, 0, 0],
     [90, 18, 51, -78, 2, 98, -11, 74, -1, -7, 45, 70, 88, 1, 0, 0, 1, 0, 0, 0],
     [-3, 95, -57, -18, -33, 100, -18, 85, 84, -17, 42, 29, -52, 74, 0, 0, 0, 1, 0, 0],
     [47, 24, -19, 89, -74, 77, 13, 60, 83, 66, -17, -24, -3, 58, 0, 0, 0, 0, 1, 0],
     [-75, 43, 46, 72, 21, -99, 97, 92, 82, -25, 41, 50, -5, 57, 0, 0, 0, 0, 0, 1]
     ])

#coeficientes de la función objetivo
c = np.array([44,18,60,27,-79,-51,92,-78,-69,-35,26,-56,-10,-55,0,0,0,0,0,0])

#términos independientes de las restricciones
b = np.array([64, 537, 55, 292, 1017, 6, 441, 312, 381, 398])

m=len(A) # num de restricciones
n=len(A[0]) # num de variables

id=np.identity(m) #matriz identidad (la usamos para crear la A de la fase I)
zeros=np.zeros(m) 


# Fase II

Ahora que tenemos una SBF del problema inicial podemos ejecutar la Fase II, que en realidad son los mismos pasos que hemos estado siguiendo en la fase 1 pero con la A y la c del problema original

**SBF inicial:**
* Índices variables básicas: [ 8  4  1  2  3 11 10  0  5  7 ]
* Índices variables básicas: [ 6  9 12 13 14 15 16 17 18 19 ] //eliminamos las variables artificiales

**1. Inicialización**

In [ ]:
indices_basicas = np.array([ 8, 4 , 1 , 2 , 3 ,11 ,10 , 0 , 5 , 7])
indices_no_basicas = np.array([ 6 , 9 , 12 ,13 ,14 ,15 ,16 ,17 ,18 ,19]) #eliminamos las variables artificiales

#Inicializamos B y An
filas=np.array(range(m))
B = A[np.ix_(filas,indices_basicas)]
An = A[np.ix_(filas,indices_no_basicas)]

#Inicializamos cb y cn
cb = c[indices_basicas]
cn = c[indices_no_basicas]

#Calculamos la inversa de B
B_inv = np.linalg.inv(B)

#Calculamos Xb
Xb = B_inv @ b

z = cb @ Xb

if all(val>=0 for val in Xb):
    print('Es SBF')


Es SBF


# 1a ITERACIÓN

**2. Identificación de SBF óptima y selección de variable de entrada:**
1. Calcular los costes reducidos 𝑟
2. Si 𝑟 ≥ 0 entonces la actual SBF es óptima → STOP. Sino, seleccionar una variable no básica 𝑞 con 𝑟𝑞′ < 0 (variable de entrada)


In [166]:
# r = Cn - Cb * inv(B) * An
cb_x_invB = cb @ B_inv  # Multiplicación de cb por la inversa de B
cb_x_invB_An = cb_x_invB @ An  # Multiplicación del resultado anterior por An
r = cn - cb_x_invB_An 

print(f"r = {r}")

# Selección de la variable de entrada usando la regla de Bland
# elegimos la variable no básica de entrada con el subindice más pequeño con coste negativo 
optimo = True
q_idx = None
for i in range(len(r)):
    # Como ordenamos las variables no básicas por los subíndices, elegimos el primer valor negativo. Si no hay valores negativos, quiere decir que es optimo
    if r[i]<-1e-10:
        q_idx=i #subindice de la variable de entrada
        optimo = False
        break

if optimo:
    print("SBF optimo fase II encontrado")
    
    
else:
    print("No es el óptimo, seguimos buscando")
q = indices_no_basicas[q_idx]
print(z)
print(f"q = {q}" if q_idx is not None else "")            

r = [-290.76772351 -371.09167545  506.36809024   30.65557113    3.74177719
    1.24492671   -4.40677378    3.10429366   -6.92626358   -2.74312183]
No es el óptimo, seguimos buscando
-35.17910078240501
q = 6


**3. Cálculo de un DBF de descenso:**
1. Calcular 𝑑𝐵 = −inv(𝐵)*𝐴𝑞 (DBF asociada a 𝑥𝑞)
2. Si 𝑑𝐵 ≥ 0 ⟹ DBF de descenso no acotada⟹ 𝑃𝐿 no acotado → STOP

In [167]:
Aq = A[:, q]  # Extraemos la columna correspondiente a xq en A
db = -B_inv @ Aq  # Calculamos la dB

# Si todos los valores en db son mayores o iguales a 0, el problema no está acotado
if np.all(db >= 0):  
    print('(PL) no acotado')  
else:
    print("(PL) acotado")

print(f"Aq = {Aq}")
print(f"db = {db}")


(PL) acotado
Aq = [ 54  32  56  56  67 -11 -11 -18  13  97]
db = [ 0.59242459  1.45206249 -0.91774907 -1.35634437 -0.44581336  0.58851968
 -1.66615427  0.30878153  0.27906446  0.51688937]


**4. Cálculo de la longitud de paso máximo y selección de la variable de sálida:**
1. Escoger la 𝜃* 
2. Variable básica de salida: 𝐵 𝑝 tal que 𝜃* = −𝑥ℬ 𝑝 /𝑑ℬ(𝑝)

In [168]:
# Calculamos las 𝜃 (donde db(i) és negativo) y escogemos la mínima, 
# si db(i) es postivo, ponemos un valor infinito para no escoger esa 𝜃
thetas_lst = np.where(db < -1e-10, -Xb / db, np.inf) 
print(f"𝜃 = {thetas_lst}")
theta = np.min(thetas_lst) # Escogemos el valor mínimo 
p = np.argmin(thetas_lst)
print(f"𝜃* = {theta}")
print(f"p = {p}")

𝜃 = [       inf        inf 0.92809629 0.98673063 4.65087938        inf
 1.23726025        inf        inf        inf]
𝜃* = 0.9280962939319488
p = 2


**5. Actualizaciones y cambio de base:**
1. Actualizar las variables básicas y la función objetivo: x𝐵 ≔ xB + 𝜃𝑑𝐵, 𝑥𝑞 ≔ 𝜃 ; 𝑧 = 𝑧 + 𝜃*𝑟𝑞
2. Actualizar los conjuntos ℬ y 𝒩: ℬ:=ℬ∖{𝐵(𝑝)} ∪ {𝑞} , 𝒩 ≔ 𝒩 ∖ {𝑞} ∪ {𝐵(p)}

In [169]:
# Actualizamos índices para las variables básicas
temp = indices_basicas[p]
indices_basicas[p] = q

# Actualizar índices para las variabels no básicas (quitar q y añadir el que salió)
indices_no_basicas = np.array([i for i in range(A.shape[1]) if i not in indices_basicas])

# Actualizar las matrices B y An con los nuevos ínidices
B = A[:, indices_basicas]
An = A[:, indices_no_basicas]

#Actualizar cb y cn
cb = c[indices_basicas]
cn = c[indices_no_basicas]

# Actualizar Xb 
Xb_actual = np.zeros(m)
for i in range(m):
    if i == p:
        Xb_actual[i] = theta
    else:
        Xb_actual[i] = Xb[i] + theta * db[i]
Xb = Xb_actual

# Verificar factibilidad
if np.any(Xb_actual < -1e-10):  
    print("Solución no factible")

# Actualizar z
z_nuevo = float(z) + r[q_idx] * theta
#Comprovar que la z nueva es menor que la z anterior
if (z_nuevo < z):
    print(f"La z nueva mejora -> z_nueva = {z_nuevo}  z_anterior = {z}")
else:
    print(f"La z nueva NO mejora -> z_nueva = {z_nuevo}  z_anterior = {z}")
z = z_nuevo

#Actualización de la B_inv

# Calcular y_k = B_inv * a_k (columna que entra)
a_k = A[:, q]       # Se obtiene la columna 'q' de la matriz A (variable entrante)
y_k = B_inv @ a_k   # Se calcula y_k = B_inv * a_k

# Construir la matriz de transformación E
E = np.eye(m)               # Inicializamos E como una matriz identidad de tamaño m×m
E[:, p] = -y_k / y_k[p]    # Reemplazamos la columna 'p' de E con -y_k / y_k[p]
E[p, p] = 1 / y_k[p]       # Ajustamos el elemento diagonal E[p,p] para pivotear

# Actualizar B_inv usando E
B_inv_actualizada = E @ B_inv

#Comprobamos si se actualiza bien la inversa de B
if np.allclose(np.linalg.inv(B), B_inv_actualizada):
    print("Inversa de B correctamente actualizada")
    B_inv = B_inv_actualizada
else:
    print("Inversa de B no se actualiza correctamente, volvemos a calcular la inversa desde cero")
    B_inv = np.linalg.inv(B)

La z nueva mejora -> z_nueva = -305.0395473659028  z_anterior = -35.17910078240501
Inversa de B correctamente actualizada


**6. Ir a 2.**


# N-sima iteración:
**Si continuamos ejecutando el simplex, estas son los resultados de las iteraciones futuras:**

**Iteraciones de la Fase II**

| Iteración | q | B(p)  | θ*            | z             |
|----------|-------------|----------------|---------------|---------------|
| 12       | 6           | 1              | 0.928         | -35.179101    |
| 13       | 9           | 2              | 0.294         | -305.039547   |
| 14       | 1           | 6              | 0.124         | -344.318142   |
| 15       | 13          | 1              | 0.241         | -395.391996   |
| 16       | 12          | 10             | 0.335         | -450.020698   |
| 17       | 1           | 5              | 0.398         | -564.286290   |
| 18       | 2           | 3              | 0.163         | -582.095550   |
| 19       | 14          | 1              | 85.639        | -587.906477   |
| 20       | 16          | 0              | 52.632        | -590.007854   |
| 21       | 3           | 2              | 0.006         | -620.293537   |
| 22       | 15          | 12             | 273.514       | -621.063902   |
| 23       | 18          | 3              | 12.861        | -625.173317   |
| 24       | 5           | 13             | 0.372         | -626.793898   |
| 25       | 12          | 9              | 0.315         | -638.437594   |
| 26       | 19          | 11             | 103.364       | -645.740557   |
| 27       | 0           | 16             | 1.119         | -670.629405   |
| 28       | 13          | 15             | 1.288         | -672.791338   |
| 29       | 9           | 0              | 0.009         | -702.939936   |
| 30       | 15          | 18             | 118.532       | -702.986645   |
| 31      | -         | -            | -   | -703.648371   |

**Solución óptima encontrada en la iteración 31**  
**z final:** z = -703.648371  
**Fin del simplex primal**